# CONUAR Python-Test

#Objetivos

1.   Object detection
  *   Detectar y marcar con un recuadro la patente del auto de la imagen "Auto.jpg".
  *   Ademas de recuadrar la patente, blurearla (Opcional)
2.   Face detection
  *   Detectar y marcar con un recuadro el rostro de la mujer de la imagen "Mujer.jpg"
1.   Object traking (Opcional)
  *   Desarrolla un programa que tome la imagen de la webcam, detecte si hay un rostro, lo recuadre y lo siga cuando se mueve

#Archivos

https://drive.google.com/drive/folders/1bqrQOZtEbyaq-WZVnsF_vmdNmSjgzMgU?usp=sharing

#Instalación de requerimientos

In [ ]:
!pip uninstall opencv-python-headless
!pip install opencv-python-headless==4.5.2.52
!pip install easyocr
!pip install imutils

     |████████████████████████████████| 38.2 MB 1.2 MB/s 
     |████████████████████████████████| 63.6 MB 57 kB/s 


#Importar dependencias

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr

#Object Detection

In [ ]:
#@title Lectura de imagen y escala de grises
img = cv2.imread('Auto.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))

In [ ]:
#@title Aplicar filtro y buscar bordes
bfilter = cv2.bilateralFilter(gray, 11, 17, 17) #Noise reduction
edged = cv2.Canny(bfilter, 30, 200) #Edge detection
plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

In [ ]:
#@title Buscar contorno y aplicar mascara
keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(keypoints)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

In [ ]:
location = None
for contour in contours:
  approx = cv2.approxPolyDP(contour, 10, True)
  if len(approx) == 4:
    location = approx
    break

In [ ]:
mask = np.zeros(gray.shape, np.uint8)
new_image = cv2.drawContours(mask, [location], 0, 255, -1)
new_image = cv2.bitwise_and(img, img, mask=mask)

In [ ]:
(x,y) = np.where(mask==255)
(x1, y1) = (np.min(x), np.min(y))
(x2, y2) = (np.max(x), np.max(y))
cropped_image = img[x1:x2+1, y1:y2+1]
cropped_image = cv2.blur(cropped_image, (5,5))

In [ ]:
plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))

In [ ]:
#@title Recuadrar y renderizar resultado
img[x1:x2+1, y1:y2+1] = cropped_image
cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0,255,0), 2)

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

#Face Detection

In [ ]:
#@title Cargar Cascade Classifier
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
#@title Lectura de imagen y escala de grises
img = cv2.imread('Mujer.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
#@title Deteccion de rostro
faces = face_cascade.detectMultiScale(gray, 1.1, 4)

In [ ]:
#@title Recuadrar
for (x, y, w, h) in faces:
  cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 5)

In [ ]:
#@title Renderizar resultados
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

#Object Tracking

(Solo correr como script)

In [ ]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture('Webcam.mp4')

#cap = cv2.VideoCapture(0)
#cap.set(cv2.CAP_PROP_FPS, 30)

while True:
  _, img = cap.read()

  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  faces = face_cascade.detectMultiScale(gray, 1.1, 4)

  for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 3)
  
  cv2.imshow('VideoCapture', img)

  if cv2.waitKey(1) == ord('q'):
    break
    
cap.realese()
cv2.destroyAllWindows()